In [ ]:
from langchain.prompts import FewShotPromptTemplate
from langchain.llms import OpenAIChat
from langchain.callbacks import get_openai_callback
from langchain.chains import LLMChain
import requests
import datetime



%reload_ext lab_black

In [ ]:
# url = 'https://newsapi.org/v2/sources'
# params = {
#     'apiKey': NEWS_API_KEY
# }

# response = requests.get(url, params=params)

# sources = response.json()['sources']

# for source in sources:
#     print(source['id'], '-', source['name'])

In [ ]:
finance_regulatory_bodies = [
    "Consumer Financial Protection Bureau",
    "CFPB",
    "Federal Reserve System",
    "The Fed",
    "Securities and Exchange Commission",
    "SEC",
    "Commodity Futures Trading Commission",
    "CFTC",
    "National Credit Union Administration",
    "NCUA",
    "Office of the Comptroller of the Currency",
    "OCC",
    "Financial Industry Regulatory Authority",
    "FINRA",
    "Financial Stability Oversight Council",
    "FSOC",
    "Internal Revenue Service",
    "IRS",
]

finance_regulatory_bodies = " OR ".join(
    [f"'{s.lower()}'" for s in finance_regulatory_bodies]
)

query = f"(regulation OR compliance OR legislation) AND (cfpb OR finra OR OCC)"
# "law OR regulation OR policy OR compliance OR legislation OR government oversight"
# categories = "business,technology,general"    # "category": categories,
from_date = datetime.datetime.now() - datetime.timedelta(days=7)
to_date = datetime.datetime.now()

url = "https://newsapi.org/v2/everything"
params = {
    "q": query,
    "from": from_date,
    "to": to_date,
    "language": "en",
    "sortBy": "publishedAt",
    "pageSize": 100,
    "page": 1,
    "apiKey": NEWS_API_KEY,
}

response = requests.get(url, params=params)

In [ ]:
articles = response.json()["articles"]

In [ ]:
for article in articles:
    print(article["title"], "-", article["source"]["name"], "-", article["publishedAt"])

How about some more federal laws to fix banking? - Americanthinker.com - 2023-03-18T05:00:00Z
Bragar Eagel & Squire, P.C. Reminds Investors That Class Action Lawsuits Have Been Filed Against Block, Atlassian, PLDT, and Global Payments and Encourages Investors to Contact the Firm - GlobeNewswire - 2023-03-18T01:00:00Z
Innovating during a regulatory wave - VentureBeat - 2023-03-17T20:07:00Z
Consumer Watchdog Wants to Know Who's Watching Your Internet Behavior - Investopedia - 2023-03-17T15:26:26Z
Regulators Shut Down Banks, Raising Questions About Neutrality - Forbes - 2023-03-17T11:30:00Z
Moomoo's Continuous Innovation Journey on AWS - PRNewswire - 2023-03-17T11:00:00Z
Regulatory Failure 101: What the Collapse of Silicon Valley Bank Reveals - ProPublica - 2023-03-17T11:00:00Z
So Where Were the Regulators? - The Atlantic - 2023-03-17T11:00:00Z
Buy-backs of shares in CoinShares International Limited - Marketscreener.com - 2023-03-17T08:01:01Z
Buy-backs of shares in CoinShares Internationa

# Langchain

In [ ]:
llm = OpenAIChat(temperature=0.0)

In [ ]:
# Also, I don't need any article that talks about regulation but rather something that discusses a regulatory change that might impact my work

In [ ]:
prefix = """You are a news recommender and your job is to look at an article and decide if I would find it useful or not.
I work in a bank/credit card company and I'm looking for news articles that deal with changes in the regulatory environment. 
It could be articles about regulatory authorities like CFPB or OCC announcing changes in regulation or a new focus areas. 
Or articles like the president or senate or government announcing new regulations. 
Or peer financial institutions getting fined for violations. 
To reiterate I'm especially looking for articles in the finance, fin-tech, banking and credit card sector. 
**I don't need think pieces or opinion pieces or editorials that talk about the consequences of a regulation but just the ones that talk about objective changes in the regulatory environment that might impact my work**
I'm going to pass in a few headlines. Can you just respond with "relevant" or "not relevant" for the headlines I send you?"""

examples = [
    {
        "headline": "CFPB Launches Inquiry Into the Business Practices of Data Brokers",
        "publisher": "CFPB",
        "answer": "relevant",
    },
    {
        "headline": "OCC Issues Prohibition Order, Fines Former Wells Fargo Executive $17 Million in Settlement",
        "publisher": "NYT",
        "answer": "relevant",
    },
    {
        "headline": "Innovating during a regulatory wave",
        "publisher": "Venture beat",
        "answer": "not relevant",
    },
]

In [ ]:
example_prompt = PromptTemplate(
    input_variables=["headline", "publisher", "answer"],
    template="Headline: {headline}\nPublisher: {publisher}\nAnswer: {answer}",
)

In [ ]:
prompt = FewShotPromptTemplate(
    prefix=prefix,
    examples=examples,
    example_prompt=example_prompt,
    suffix="Headline: {headline}\nPublisher: {publisher}\nAnswer: ",
    input_variables=["headline", "publisher"],
)

In [ ]:
print(
    prompt.format(
        **{
            "headline": "CFPB Launches Inquiry Into the Business Practices of Data Brokers",
            "publisher": "CFPB",
        }
    )
)

You are a news recommender and your job is to look at an article and decide if I would find it useful or not.
I work in a bank/credit card company and I'm looking for news articles that deal with changes in the regulatory environment. 
It could be articles about regulatory authorities like CFPB or OCC announcing changes in regulation or a new focus areas. 
Or articles like the president or senate or government announcing new regulations. 
Or peer financial institutions getting fined for violations. 
To reiterate I'm especially looking for articles in the finance, fin-tech, banking and credit card sector. 
**I don't need think pieces or opinion pieces or editorials that talk about the consequences of a regulation but just the ones that talk about objective changes in the regulatory environment that might impact my work**
I'm going to pass in a few headlines. Can you just respond with "relevant" or "not relevant" for the headlines I send you?



Headline: CFPB Launches Inquiry Into the B

In [ ]:
news_predictor_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
news_predictor_chain.run(
    {
        "headline": "Ray Dalio Commentary: What I Think About the Silicon Valley Bank Situation",
        "publisher": "Yahoo Entertainment",
    }
)

{'headline': 'Ray Dalio Commentary: What I Think About the Silicon Valley Bank Situation',
 'publisher': 'Yahoo Entertainment',
 'text': 'not relevant'}

In [ ]:
news_predictor_chain.run(
    {
        "headline": "Financial Regulators, Black History and Epistemic Capital",
        "publisher": "Harvard School of Engineering",
    }
)

{'headline': 'Financial Regulators, Black History and Epistemic Capital',
 'publisher': 'Harvard School of Engineering',
 'text': 'not relevant'}

In [ ]:
news_predictor_chain(
    {
        "headline": "Agencies Issue Joint Statement on Crypto-Asset Risks to Banking Organizations",
        "publisher": "OCC",
    }
)

{'headline': 'Agencies Issue Joint Statement on Crypto-Asset Risks to Banking Organizations',
 'publisher': 'OCC',
 'text': 'relevant'}

In [ ]:
news_predictor_chain.run(
    {
        "headline": "Acting Comptroller of the Currency Testifies on Regulatory Priorities",
        "publisher": "The Washington Post",
    }
)

{'headline': 'Acting Comptroller of the Currency Testifies on Regulatory Priorities',
 'publisher': 'The Washington Post',
 'text': 'relevant'}

In [ ]:
news_predictor_chain.run(
    {
        "headline": "The Impact Of Evolving Cloud Regulations On Financial Services",
        "publisher": "Forbes",
    }
)

{'headline': 'The Impact Of Evolving Cloud Regulations On Financial Services',
 'publisher': 'Forbes',
 'text': 'relevant'}

In [ ]:
sum([1, 2])

3